In [ ]:
from skimage.io import imread
import matplotlib.pyplot as plt

import os,sys,inspect

import requests     #Traer el fichero de internet
import json         # Trabaja con ficheros json
import pandas as pd # Librería para el data mining y data wrangling
import numpy as np



imagen = imread("./img/python.jpg")
#currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))  --> otra forma de encontrar el directorio del archivo, por favor, explicar diferencia.
currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

def get_root_path(n):
    path = os.getcwd() # para notebook ||| __file__ --> para .py
    for i in range(n):
        path = os.path.dirname(path)
    print(path)
    sys.path.append(path)

# n --> es el número de veces que se tiene que hacer os.path.dirname 
# si en .ipynb notebook es 5 entonces en .py usando __file__ es 6. 
get_root_path(n=5)

# Se importan las librerías de clases desde un directorio superior
import objects.human as ohu
import objects.ork as oo